# Pandas 筆記
#### 建立DataFrame
將dictionary的資結構轉換成data frame, 可以支援篩選布林值、排序、統計、相異值個數

In [1]:
import pandas as pd

dic = {
    "col 1" : [1,2,3],
    "col 2" : [10,20,30],
    "col 3" : list('xyz'),
    "col 4" : ['a','b','c'],
    "col 5" : pd.Series(range(3))
}
df = pd.DataFrame(dic)
print(df)

   col 1  col 2 col 3 col 4  col 5
0      1     10     x     a      0
1      2     20     y     b      1
2      3     30     z     c      2


In [2]:
rename_dic = {"col 1":"x", "col 2":"10x"}
df.rename(rename_dic, axis = 1)

,x,10x,col 3,col 4,col 5
0,1,10,x,a,0
1,2,20,y,b,1
2,3,30,z,c,2


### Case 1: 例如台北市107年的道路肇事案件，再進行處理
[LeeMeng, 資料科學家的 pandas 實戰手冊：掌握 40 個實用數據技巧](https://leemeng.tw/practical-pandas-tutorial-for-aspiring-data-scientists.html)

#### 可能會用到的函數:  
`.read_csv(csv_url, encoding='big5')`從網路上下載
`.apply(lambda x)`讓一個函式作用在"一維"的方向向量上  
`.applymap(lambda x)`讓函式作用在DataFrame上的每個元素  
`.map(lambda x)`讓函式作用在一個Series上的每一個元素  
`.isin()`判斷元素有沒有在Data Frame中  
`.groupby()`依照自己要的column去做分組，可以用`size()`去看每個組內的大小，可以再利用aggregation function來計算該群集合的統計值如`.mean()`、`.sum()`、`std()`......  
`.sort_values()`依據不同的標籤做資料排序  
`.iloc()`以數字為index來做選擇
`.reset_index()`default設定會直接產生出一個新的index行在舊的index的左邊  
`.drop('index', axis=1)`刪除某個行
`.fillna(0)`將 DataFrame 裡頭所有不存在的值設為`0`  

#### Concatenation(接續)與Merge(合併):  
接續（concatenation）是指在第一個 Dataframe 的最後一列後接續第二個 datarame

`pd.concat([df1,df2], ignore_index=True)    # create a new integer index`  
選項`join='inner'`可以使得兩個Dataframe的共同columns才會留下來

合併(merge)則是pandas 會直接尋找兩個 dataframe 共同的 columns 並依據其中的值來合併
`pd.merge(df1,df2)    

In [3]:
base_url = "https://data.taipei/api/getDatasetInfo/downloadResource?id={}&rid={}"
_id = "2f238b4f-1b27-4085-93e9-d684ef0e2735"
rid = "ea731a84-e4a1-4523-b981-b733beddbc1f"
csv_url = base_url.format(_id, rid)
print(csv_url)
df_raw = pd.read_csv(csv_url, encoding='big5')

# 複製一份
df = df_raw.copy()

# 計算不同區不同性別的死亡、受傷人數
df['區序'] = df['區序'].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
df = (df[df['性別'].isin([1,2])]
      .groupby(['區序','性別'])[['死亡人數','受傷人數']]
      .sum()
      .reset_index()
      .sort_values('受傷人數'))
df['性別'] = df['性別'].apply(lambda x: '男性' if x == 1 else '女性')
df = df.reset_index().drop('index', axis=1)

# 顯示結果
display(df_raw.head())
display(df)

https://data.taipei/api/getDatasetInfo/downloadResource?id=2f238b4f-1b27-4085-93e9-d684ef0e2735&rid=ea731a84-e4a1-4523-b981-b733beddbc1f


,發生年,發生月,發生日,發生時,發生分,處理別,區序,肇事地點,死亡人數,受傷人數,當事人序,車種,性別,年齡,受傷程度,4天候,7速限,8道路型態,9事故位置
0,107,3,29,15,54,2,01大同區,大同區民權西路108號,0,1,1,B03,2,41.0,3.0,8,50,14.0,9.0
1,107,3,29,15,54,2,01大同區,大同區民權西路108號,0,1,2,C03,2,58.0,2.0,8,50,14.0,9.0
2,107,1,7,17,42,2,01大同區,大同區重慶北路2段與南京西路口,0,1,1,B01,1,59.0,3.0,6,40,4.0,2.0
3,107,1,7,17,42,2,01大同區,大同區重慶北路3段與南京西路口,0,1,2,C03,1,18.0,2.0,6,40,4.0,2.0
4,107,1,14,9,56,2,01大同區,大同區承德路3段與民族西路口,0,1,1,C03,1,20.0,2.0,8,50,4.0,1.0


,區序,性別,死亡人數,受傷人數
0,南港區,女性,2,799
1,萬華區,女性,2,1084
2,松山區,女性,30,1146
3,大同區,女性,3,1336
4,士林區,女性,3,1660
5,南港區,男性,7,1861
6,大安區,女性,9,1969
7,信義區,女性,3,2022
8,中正區,女性,7,2023
9,文山區,女性,1,2106


In [14]:
import chartify
ch = chartify.Chart(
    x_axis_type='categorical', 
    y_axis_type='categorical')

ch.plot.heatmap(
    data_frame=df, 
    y_column='性別', 
    x_column='區序', 
    color_column='受傷人數', 
    text_column='受傷人數', 
    color_palette='Reds', 
    text_format='{:,.0f}')

(ch.set_title('2017 年台北市各行政區交通意外受傷人數')
 .set_subtitle('性別分計')
 .set_source_label("Data.Taipei")
 .axes.set_xaxis_label('性別')
 .axes.set_yaxis_label('行政區'))
ch.show('html')

### 條件選取數據
#### 以鐵達尼號(kaggle)當例子，實作masking
有時dataframe太大可以只選擇讀幾個column就好

In [30]:
cols = ['PassengerId','Survived','Pclass','Sex','Age','Parch','Ticket','Fare','Cabin','Embarked']
df = pd.read_csv('http://bit.ly/kaggletrain', usecols=cols)
df.info(memory_usage="deep")
display(df.head())    # 只顯示部分數據

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Sex            891 non-null object
Age            714 non-null float64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(4), object(4)
memory usage: 238.4 KB


,PassengerId,Survived,Pclass,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,male,22.0,0,A/5 21171,7.2500,NaN,S
1,2,1,1,female,38.0,0,PC 17599,71.2833,C85,C
2,3,1,3,female,26.0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,female,35.0,0,113803,53.1000,C123,S
4,5,0,3,male,35.0,0,373450,8.0500,NaN,S


將年紀超過70歲的男性找出來

In [31]:
(df[(df.Sex == 'male') & (df.Age > 70)]
    .style
    .applymap(lambda x: 'background-color: rgb(153, 255, 51)', subset=pd.IndexSlice[:, 'Sex':'Age']))

,PassengerId,Survived,Pclass,Sex,Age,Parch,Ticket,Fare,Cabin,Embarked
96,97,0,1,male,71,0,PC 17754,34.6542,A5,C
116,117,0,3,male,70.5,0,370369,7.75,nan,Q
493,494,0,1,male,71,0,PC 17609,49.5042,nan,C
630,631,1,1,male,80,0,27042,30,A23,S
851,852,0,3,male,74,0,347060,7.775,nan,S
